In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from datetime import datetime

browser = webdriver.Chrome()

base_urls = [
    "https://www.pisos.com/venta/pisos-madrid/",
    "https://www.pisos.com/venta/pisos-la_rioja/",
    "https://www.pisos.com/venta/pisos-navarra_nafarroa/",
    "https://www.pisos.com/venta/pisos-asturias/",
    "https://www.pisos.com/venta/pisos-cantabria/",
    "https://www.pisos.com/venta/pisos-murcia/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/",
    "https://www.pisos.com/venta/pisos-las_palmas/",
    "https://www.pisos.com/venta/pisos-santa_cruz_de_tenerife/",
    "https://www.pisos.com/venta/pisos-badajoz/",
    "https://www.pisos.com/venta/pisos-caceres/",
    "https://www.pisos.com/venta/pisos-guipuzcoa_gipuzkoa/",
    "https://www.pisos.com/venta/pisos-vizcaya_bizkaia/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-valencia/",
    "https://www.pisos.com/venta/pisos-alicante/",
    "https://www.pisos.com/venta/pisos-teruel/",
    "https://www.pisos.com/venta/pisos-huesca/",
    "https://www.pisos.com/venta/pisos-zaragoza/",
    "https://www.pisos.com/venta/pisos-girona/",
    "https://www.pisos.com/venta/pisos-lleida/",
    "https://www.pisos.com/venta/pisos-barcelona/",
    "https://www.pisos.com/venta/pisos-lugo/",
    "https://www.pisos.com/venta/pisos-pontevedra/",
    "https://www.pisos.com/venta/pisos-ourense/",
    "https://www.pisos.com/venta/pisos-a_coruna/",
    "https://www.pisos.com/venta/pisos-guadalajara/",
    "https://www.pisos.com/venta/pisos-toledo/",
    "https://www.pisos.com/venta/pisos-cuenca/",
    "https://www.pisos.com/venta/pisos-albacete/",
    "https://www.pisos.com/venta/pisos-ciudad_real/",
    "https://www.pisos.com/venta/pisos-salamanca/",
    "https://www.pisos.com/venta/pisos-avila/",
    "https://www.pisos.com/venta/pisos-segovia/",
    "https://www.pisos.com/venta/pisos-soria/",
    "https://www.pisos.com/venta/pisos-burgos/",
    "https://www.pisos.com/venta/pisos-palencia/",
    "https://www.pisos.com/venta/pisos-valladolid/",
    "https://www.pisos.com/venta/pisos-leon/",
    "https://www.pisos.com/venta/pisos-zamora/",
    "https://www.pisos.com/venta/pisos-alava_araba/",
    "https://www.pisos.com/venta/pisos-almeria/",
    "https://www.pisos.com/venta/pisos-cadiz/",
    "https://www.pisos.com/venta/pisos-castellon_castello/",
    "https://www.pisos.com/venta/pisos-cordoba/",
    "https://www.pisos.com/venta/pisos-granada/",
    "https://www.pisos.com/venta/pisos-huelva/",
    "https://www.pisos.com/venta/pisos-jaen/",
    "https://www.pisos.com/venta/pisos-malaga/",
    "https://www.pisos.com/venta/pisos-sevilla/",
    "https://www.pisos.com/venta/pisos-tarragona/",
    "https://www.pisos.com/venta/pisos-ceuta/",
    "https://www.pisos.com/venta/pisos-melilla/",
    "https://www.pisos.com/venta/pisos-islas_baleares_illes_balears/"
]

fecha_limite = datetime.strptime("2025-03-01", "%Y-%m-%d")  

def extraer_enlaces(url, browser, lista_sucia):
    browser.get(url)
    # Aceptar cookies
    try:
        browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
        sleep(1)
    except:
        pass

    inmuebles = browser.find_elements(by=By.CSS_SELECTOR, value="div.ad-preview")
    
    if not inmuebles:
        print("No hay inmuebles en esta página.")
        return False

    for inmueble in inmuebles:
        try:
            # Extraer el enlace al detalle del inmueble
            link = inmueble.find_element(by=By.CLASS_NAME, value="ad-preview__title").get_attribute("href")
            lista_sucia.append(link)
        except Exception as e:
            print(f"Error al procesar un inmueble: {e}")
    
    return True

def filtrar_por_fecha(lista_sucia, browser, fecha_limite):
    lista_limpia = []
    
    for link in lista_sucia:
        try:
            browser.get(link)
            try:
                fecha_str = browser.find_element(by=By.CSS_SELECTOR, value="p.last-update__date").text
                fecha_actualizacion = datetime.strptime(fecha_str.split("el")[1].strip(), "%d/%m/%Y")
                
                # Si la fecha es posterior a lo que pusimos, guardamos el enlace
                if fecha_actualizacion >= fecha_limite:
                    lista_limpia.append(link)
                    print(f"Enlace añadido a lista_limpia: {link} (Actualizado el {fecha_actualizacion.strftime('%d/%m/%Y')})")
                else:
                    print(f"Fecha de actualización {fecha_actualizacion.strftime('%d/%m/%Y')} es anterior a la fecha límite.")
            except Exception as e:
                print(f"No se pudo obtener la fecha de actualización de {link}: {e}")
        except Exception as e:
            print(f"Error al procesar el enlace {link}: {e}")
    
    return lista_limpia

# Procesar cada provincia
for base_url in base_urls:
    print(f"\nProcesando provincia: {base_url}")
    
    lista_sucia = []  # Limpiar lista_sucia para cada provincia
    pagina = 1
    max_pagina = 100  # Limitar a 100 páginas por provincia
    
    while pagina <= max_pagina:
        #print(f"  Procesando página {pagina}...")
        url_actual = f"{base_url}fecharecientedesde-desc/{pagina}/"
        
        if not extraer_enlaces(url_actual, browser, lista_sucia):
            print("  No se encontraron más resultados o se alcanzó el final de las páginas.")
            break

        pagina += 1

    print(f"\n  Filtrando inmuebles de la provincia...")
    lista_limpia = filtrar_por_fecha(lista_sucia, browser, fecha_limite)

    print(f"  Hay {len(lista_limpia)} inmuebles en esta provincia desde la última actualización.")

browser.quit()



Procesando provincia: https://www.pisos.com/venta/pisos-madrid/

  Filtrando inmuebles de la provincia...
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/piso-el_barral_ferial28822-52590980094_109800/ (Actualizado el 16/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/piso-palomeras_sureste28018-51694920821_100200/ (Actualizado el 16/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/piso-hortaleza_palomas28042-52565154662_105100/ (Actualizado el 16/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/casa_adosada-zona_suroeste-52554000432_101000/ (Actualizado el 16/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/duplex-centro_palacio28005-51742231035_100500/ (Actualizado el 16/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/duplex-cobena_centro_urbano-50889930043_103800/ (Actualizado el 04/03/2025)
Enlace añadido a lista_limpia: https://www.pisos.com/comprar/piso-trafalga

In [ ]:
df_csv = pd.read_csv('compras_completo_limpio.csv')
urls_csv_original = df_csv['href'].tolist()

urls_a_scrapear = [url for url in lista_limpia if url not in urls_csv_original]

if urls_a_scrapear:
    print(f"Hay {len(urls_a_scrapear)} URLs que no existen en el archivo CSV original:")

else:
    print("Todas las URLs obtenidas ya existen en el archivo CSV original")